# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv()

True

In [3]:
MODEL="gpt-4o-mini"

In [4]:
system_message = """You are an airline assistant. 
Only use the get_destination_price function when the user specifically asks about flight prices, costs, or wants to travel to a specific destination.
For general greetings, questions, or conversations that don't involve travel pricing, respond normally without using any tools."""

In [11]:
city_price_map = {"london": "$1000", "new york city": "$1200", "chicago": "$400", "california": "$900"}
def get_destination_price(destination_city: str) -> str:
    city = destination_city.lower()
    return city_price_map.get(city, "Unknown")

In [12]:
price_function = {
    "type": "function",
    "function": {
        "name": "get_destination_price",
        "description": "Get price information for a destination",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The destination city"
                }
            },
            "required": ["destination_city"],
            "additionalProperties": False
        }
    }
}

In [13]:
openai = OpenAI()

In [14]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_destination_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [19]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=[price_function])

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        print(messages)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [20]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': "You are an airline assistant. \nOnly use the get_destination_price function when the user specifically asks about flight prices, costs, or wants to travel to a specific destination.\nFor general greetings, questions, or conversations that don't involve travel pricing, respond normally without using any tools."}, {'role': 'user', 'metadata': None, 'content': 'Hi, I want to go to London', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'That sounds great! London is a wonderful city to visit. If you need information about flight prices to London, just let me know!', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'yes', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Could you please tell me the specific destination city within London you would like to know the flight price for?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'yes', 'options': None}, {'role': 'assistant', 'metadata